In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Bigdata

Mounted at /content/drive
/content/drive/MyDrive/Bigdata


In [3]:
%cd /content/drive/MyDrive/Bigdata/Datamax/

!unzip final.zip -d final/

/content/drive/MyDrive/Bigdata/Datamax
Archive:  final.zip
   creating: final/final/
  inflating: final/__MACOSX/._final  
   creating: final/final/llm_dataset/
  inflating: final/__MACOSX/final/._llm_dataset  
  inflating: final/final/llm_dataset/.part-00000-95ca6c49-3889-4db4-85cc-6b325c694ce4-c000.snappy.parquet.crc  
  inflating: final/__MACOSX/final/llm_dataset/._.part-00000-95ca6c49-3889-4db4-85cc-6b325c694ce4-c000.snappy.parquet.crc  
  inflating: final/final/llm_dataset/part-00011-95ca6c49-3889-4db4-85cc-6b325c694ce4-c000.snappy.parquet  
  inflating: final/__MACOSX/final/llm_dataset/._part-00011-95ca6c49-3889-4db4-85cc-6b325c694ce4-c000.snappy.parquet  
  inflating: final/final/llm_dataset/.part-00011-95ca6c49-3889-4db4-85cc-6b325c694ce4-c000.snappy.parquet.crc  
  inflating: final/__MACOSX/final/llm_dataset/._.part-00011-95ca6c49-3889-4db4-85cc-6b325c694ce4-c000.snappy.parquet.crc  
  inflating: final/final/llm_dataset/part-00006-95ca6c49-3889-4db4-85cc-6b325c694ce4-c000.snap

In [4]:
from datasets import Dataset
import glob
import pandas as pd

files = glob.glob("/content/drive/MyDrive/Bigdata/Datamax/final/final/llm_dataset/*.parquet")

dfs = [pd.read_parquet(f) for f in files]
df = pd.concat(dfs)

# Keep only useful cols
df = df[["id", "title", "main_text"]]

dataset = Dataset.from_pandas(df)
dataset


Dataset({
    features: ['id', 'title', 'main_text', '__index_level_0__'],
    num_rows: 2112844
})

In [5]:
def format_row(example):
    instr = "Summarize the following biomedical abstract."

    example["input_text"] = f"""### Instruction:
{instr}

### Abstract:
{example['main_text']}

### Summary:
"""
    # Target — the title is a good short summary
    example["target_text"] = example["title"]
    return example

dataset = dataset.map(format_row)
dataset = dataset.train_test_split(test_size=0.001)
dataset


Map:   0%|          | 0/2112844 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'main_text', '__index_level_0__', 'input_text', 'target_text'],
        num_rows: 2110731
    })
    test: Dataset({
        features: ['id', 'title', 'main_text', '__index_level_0__', 'input_text', 'target_text'],
        num_rows: 2113
    })
})

In [6]:
!pip install transformers accelerate bitsandbytes datasets peft sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.8 MB/s eta 0:00:00


In [7]:
from huggingface_hub import login
login()


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch

# Correct model name
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
def build_sft(example):
    example["text"] = example["input_text"] + example["target_text"]
    return example

dataset = dataset.map(build_sft, remove_columns=["input_text", "target_text", "id", "title", "main_text"])
dataset


Map:   0%|          | 0/2110731 [00:00<?, ? examples/s]

Map:   0%|          | 0/2113 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'text'],
        num_rows: 2110731
    })
    test: Dataset({
        features: ['__index_level_0__', 'text'],
        num_rows: 2113
    })
})

In [10]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=2048
    )
    return tokens

tokenized = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/2110731 [00:00<?, ? examples/s]

Map:   0%|          | 0/2113 [00:00<?, ? examples/s]

In [11]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 9,011,200 || all params: 1,109,059,584 || trainable%: 0.8125


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/Mydrive/Bigdata/tinyllama-bio-sft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    max_steps=1500,  # You can increase to 20k if running on A100
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
)


In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    data_collator=data_collator
)
trainer.train()


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    data_collator=data_collator
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
500,1.751800,1.738561
1000,1.734900,1.730361
1500,1.702300,1.727058


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1500, training_loss=1.7393874486287435, metrics={'train_runtime': 4073.0174, 'train_samples_per_second': 5.892, 'train_steps_per_second': 0.368, 'total_flos': 6.053955322719437e+16, 'train_loss': 1.7393874486287435, 'epoch': 0.011370462948398944})

In [15]:
trainer.save_model("/content/tinyllama-bio-sft")
tokenizer.save_pretrained("/content/tinyllama-bio-sft")


('/content/tinyllama-bio-sft/tokenizer_config.json',
 '/content/tinyllama-bio-sft/special_tokens_map.json',
 '/content/tinyllama-bio-sft/chat_template.jinja',
 '/content/tinyllama-bio-sft/tokenizer.model',
 '/content/tinyllama-bio-sft/added_tokens.json',
 '/content/tinyllama-bio-sft/tokenizer.json')

In [20]:
save_path = "/content/drive/MyDrive/Bigdata/tinyllama-bio-sft"

trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/Bigdata/tinyllama-bio-sft/tokenizer_config.json',
 '/content/drive/MyDrive/Bigdata/tinyllama-bio-sft/special_tokens_map.json',
 '/content/drive/MyDrive/Bigdata/tinyllama-bio-sft/chat_template.jinja',
 '/content/drive/MyDrive/Bigdata/tinyllama-bio-sft/tokenizer.model',
 '/content/drive/MyDrive/Bigdata/tinyllama-bio-sft/added_tokens.json',
 '/content/drive/MyDrive/Bigdata/tinyllama-bio-sft/tokenizer.json')

In [18]:
prompt = """### Instruction:
Answer the follwing question

### Question:
What are the side-effects of sleep inducing drugs like melatonin?

### Answer:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.4,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Answer the follwing question

### Question:
What are the side-effects of sleep inducing drugs like melatonin?

### Answer:
The side-effects of sleep inducing drugs like melatonin are generally mild and transient. However, some side-effects may occur, such as dizziness, headache, nausea, and drowsiness. These side-effects are usually mild and transient.

### Summary:
Side-effects of sleep inducing drugs.

### Author:
M.A. Kumar.

### Summary:
Side-effects of sleep inducing drugs.

### Author:
M.A. K


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_name = "therealcyberlord/TinyLlama-1.1B-Medical"

tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16  # or float32, depending on memory / GPU
)

model = PeftModel.from_pretrained(base_model, adapter_name)
model.eval()

# If you are on GPU, move model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_fea

In [6]:
# Block 2 — inference / prompt → response
prompt = """### Instruction:
Answer the Following Question

### Question:
What are the side effects of overusing drugs that

### Answer:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.4,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== MODEL OUTPUT ===")
print(generated)


=== MODEL OUTPUT ===
### Instruction:
Answer the Following Question

### Question:
I have a high fever what medicine to take

### Answer:
If you have a high fever, you should seek medical attention immediately. The best medicine to take is paracetamol (acetaminophen) or ibuprofen (meloxicam) to reduce the pain and inflammation caused by the fever. These medications are safe to take for up to 24 hours and are often recommended by doctors. It is also important to drink plenty of fluids to help keep your body hydrated and to prevent dehydration. If you experience nausea or vomiting, you should also take paracetamol or i


In [8]:
# Block 2 — inference / prompt → response
prompt = """### Instruction:
Summarize the following biomedical abstract.

### Abstract:
Supplementing with creatine is very popular amongst athletes and exercising individuals for improving muscle mass, performance and recovery. Accumulating evidence also suggests that creatine supplementation produces a variety of beneficial effects in older and patient populations. Furthermore, evidence-based research shows that creatine supplementation is relatively well tolerated, especially at recommended dosages (i.e. 3-5 g/day or 0.1 g/kg of body mass/day). Although there are over 500 peer-refereed publications involving creatine supplementation, it is somewhat surprising that questions regarding the efficacy and safety of creatine still remain. These include, but are not limited to: 1. Does creatine lead to water retention? 2. Is creatine an anabolic steroid? 3. Does creatine cause kidney damage/renal dysfunction? 4. Does creatine cause hair loss / baldness? 5. Does creatine lead to dehydration and muscle cramping? 6. Is creatine harmful for children and adolescents? 7. Does creatine increase fat mass? 8. Is a creatine 'loading-phase' required? 9. Is creatine beneficial for older adults? 10. Is creatine only useful for resistance / power type activities? 11. Is creatine only effective for males? 12. Are other forms of creatine similar or superior to monohydrate and is creatine stable in solutions/beverages? To answer these questions, an internationally renowned team of research experts was formed to perform an evidence-based scientific evaluation of the literature regarding creatine supplementation.
### Summary:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.4,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== MODEL OUTPUT ===")
print(generated)


=== MODEL OUTPUT ===
### Instruction:
Summarize the following biomedical abstract.

### Abstract:
Supplementing with creatine is very popular amongst athletes and exercising individuals for improving muscle mass, performance and recovery. Accumulating evidence also suggests that creatine supplementation produces a variety of beneficial effects in older and patient populations. Furthermore, evidence-based research shows that creatine supplementation is relatively well tolerated, especially at recommended dosages (i.e. 3-5 g/day or 0.1 g/kg of body mass/day). Although there are over 500 peer-refereed publications involving creatine supplementation, it is somewhat surprising that questions regarding the efficacy and safety of creatine still remain. These include, but are not limited to: 1. Does creatine lead to water retention? 2. Is creatine an anabolic steroid? 3. Does creatine cause kidney damage/renal dysfunction? 4. Does creatine cause hair loss / baldness? 5. Does creatine lead to d